In [1]:
#import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
from   ipywidgets import interactive, fixed
import matplotlib.pyplot as plt
from   model.cnn import CNN
import numpy as np
import pandas as pd
import random
from   source.metrics import roc_auc
from   source.preprocessing import get_list_of_patients
from   source.dataloader import ValidationDataGenerator
%config Completer.use_jedi = False

#### Settings

In [2]:
data_dir = "preprocessed-data"

# Repeated runs
n_ensemble = 50

# Data
raw_image_size = (100,100,100)

# Neural network
patch_size = (50,50,50)

# Data, second channel and gaussian blob
n_channels = 1

# Fix global seed (as good as possible)
seed = 42
np.random.seed(seed)
random.seed(seed)

#### Patients

In [3]:
patients = get_list_of_patients(data_dir)

In [4]:
patients

['/home/philipp/Projects/deep-learning-ct-colonography-public/preprocessed-data/001']

In [5]:
df_info = pd.read_csv('ct_info.csv')
df_info

,patient,polyp,histopathology,class_label,position,ct,segmentation
0,1,1,regular,benign,prone,demo-cts/demo_ct_001.npy,demo-segs/demo_seg_001.npy


#### Keras datagenerator (for testing)

In [6]:
data_generator = ValidationDataGenerator(data=patients,
                                         batch_size=len(patients),
                                         patch_size=patch_size,
                                         n_channel=1,
                                         num_threads=1,
                                         shuffle=False)

#### Check test batches

In [7]:
def plotter_batch(batch, sample_nr, channel, slice_x, slice_y, slice_z, cmap, reverse_cmap):
    image = batch[0]
    label = batch[1]
    
    # Colormap
    color_map = plt.cm.get_cmap(cmap)
    if reverse_cmap:
        color_map = color_map.reversed()
    
    print('CT Image     {:d}: {} [{:.2f}, {:.2f}]'.format(sample_nr, image[sample_nr,:,:,:,0].shape, np.amin(image[sample_nr,:,:,:,0]), np.amax(image[sample_nr,:,:,:,0])))
    if channel==2:
        print('2nd channel  {:d}: {} [{:.2f}, {:.2f}]'.format(sample_nr, image[sample_nr,:,:,:,1].shape, np.amin(image[sample_nr,:,:,:,1]), np.amax(image[sample_nr,:,:,:,1])))
    print('Label        {}'.format(label[sample_nr]))
    
    fig, ax = plt.subplots(1, 3, figsize=(21,7))
    ax[0].imshow(image[sample_nr,slice_x,:,:,channel], cmap=color_map)
    ax[1].imshow(image[sample_nr,:,slice_y,:,channel], cmap=color_map)
    ax[2].imshow(image[sample_nr,:,:,slice_z,channel], cmap=color_map)
    plt.show()

In [8]:
test_batch   = data_generator[0]
X_test_batch = test_batch[0]
y_test_batch = test_batch[1]
print('X_test_batch:', X_test_batch.shape, ', y_test_batch:', y_test_batch.shape)

patient_files ['/home/philipp/Projects/deep-learning-ct-colonography-public/preprocessed-data/001/boxedct_pat_1_segid_1.npy', '/home/philipp/Projects/deep-learning-ct-colonography-public/preprocessed-data/001/polypseg_pat_1_segid_1.npy']
patient_data_ct (100, 100, 100)
X_test_batch: (1, 50, 50, 50, 1) , y_test_batch: (1,)


In [9]:
interactive(plotter_batch,
            batch        = fixed(test_batch),
            sample_nr    = (0,X_test_batch.shape[0]-1),
            channel      = (0,X_test_batch.shape[4]-1),
            slice_x      = (0,X_test_batch.shape[1]-1),
            slice_y      = (0,X_test_batch.shape[2]-1),
            slice_z      = (0,X_test_batch.shape[3]-1),
            cmap         = ["gist_yarg", "cool", "inferno", "magma", "plasma", "viridis"],
            reverse_cmap = [True, False])

interactive(children=(IntSlider(value=0, description='sample_nr', max=0), IntSlider(value=0, description='chan…

#### Model

In [ ]:
model = ResNet18_3D_Dropout(input_shape=(50, 50, 50, num_modalities), classes=1, dropout=0.1, mc=False)
model.summary()

### Testing

In [ ]:
predictions = []

for run in range(n_ensemble):
    
    print('\nRun #{:d}'.format(run))
    
    ##############################
    ######### CNN Model ##########
    ##############################
    
    # Load trained weights from disk
    pretrained_weights = 'weights/ensemble/{:s}_{:s}'.format(model_name, str(run+1))
    
    # Load trained weights into model
    print('\nLoad weights: {:s}'.format(pretrained_weights))
    model.load_weights(pretrained_weights)
    
    #########################################
    ######### Validation & Testing ##########
    #########################################
    
    # Model predictions
    predictions_run = np.asarray(model.predict(test_data_generator))
    
    # Store predictions
    predictions.append(predictions_run)
    
predictions = np.asarray(predictions)

### Evaluation

In [ ]:
test_data_generator_2 = RVCValidDataGenerator3D(data=patients,
                                                batch_size=len(patients),
                                                patch_size=patch_size,
                                                num_modalities=num_modalities,
                                                dataset=dataset,
                                                second_channel=second_channel,
                                                sigma=sigma,
                                                hu_range=[-450,650],
                                                num_threads=20,
                                                one_hot=False,
                                                shuffle=False)
test_batch_2 = test_data_generator_2[0]
X_test       = test_batch_2[0]

In [ ]:
# Ground truth
y_true = np.asarray(test_data_generator[0][1])
y_true.shape

In [ ]:
# Ensemble predictions
y_pred_ensemble = np.mean(predictions.squeeze(), axis=0)
y_pred_ensemble.shape

#### ROC-AUC

In [ ]:
ensemble_roc_auc = roc_auc(y_true, y_pred_ensemble).numpy()
print('ROC_AUC = {:.2f}'.format(ensemble_roc_auc))

In [ ]:
# Save results
results = np.concatenate([np.expand_dims(y_pred_ensemble, -1), np.expand_dims(y_true, -1)], 1)
results.shape

In [ ]:
#results_file = 'results/ensemble/external_validation_set/results_{:s}.npy'.format(model_name)
#print('Save results: {:s}'.format(results_file))
#np.save(results_file, results)

### GradCAM

In [ ]:
def loss(output):
    loss_list  = [output[i][0] for i in range(len(patients))]
    loss_tuple = tuple(loss_list)
    return loss_tuple # (output[0][true_class[0]], output[1][true_class[1]], ...)

def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

In [ ]:
model.layers[47].name

In [ ]:
from tf_keras_vis.gradcam import GradcamPlusPlus
from tf_keras_vis.utils import normalize

cams = []

for run in range(n_runs_total):
    
    print('\nRun #{:d}'.format(run))
    
    ##############################
    ######### CNN Model ##########
    ##############################
    
    # Load trained weights from disk
    pretrained_weights = 'weights/ensemble/{:s}_{:s}'.format(model_name, str(run+1))
    
    # Load trained weights into model
    print('\nLoad weights: {:s}'.format(pretrained_weights))
    model.load_weights(pretrained_weights)
    
    ############################
    ######### GradCAM ##########
    ############################
    
    # Create Gradcam object
    gradcam = GradcamPlusPlus(model,
                              model_modifier=model_modifier,
                              clone=False)

    # Generate heatmap with GradCAM form first neuron
    cam_run = gradcam(loss,
                      X_test,
                      penultimate_layer=47, # model.layers number
                      seek_penultimate_conv_layer=False)
    cam_run = normalize(cam_run)
    
    # Store GradCAM
    cams.append(cam_run)
    
cams = np.asarray(cams)
cams.shape

In [ ]:
cam_ensemble      = np.sum(cams, axis=0)
norm_cam_ensemble = np.max(cam_ensemble.reshape(cam_ensemble.shape[0],-1),axis=1)

for i in range(norm_cam_ensemble.shape[0]):
    factor          = np.max(cam_ensemble[i])
    cam_ensemble[i] = np.divide(cam_ensemble[i], factor)

cam_ensemble = np.expand_dims(cam_ensemble, 0)
cam_ensemble.shape

#### Plot GradCAM

In [ ]:
seg_data_generator = RVCValidDataGenerator3D(data=patients,
                                             batch_size=len(patients),
                                             patch_size=patch_size,
                                             num_modalities=2,
                                             dataset=dataset,
                                             second_channel='segmentation',
                                             sigma=sigma,
                                             num_threads=20,
                                             one_hot=False,
                                             shuffle=False)


X_seg = seg_data_generator[0][0][...,1]
len(X_seg)

In [ ]:
from matplotlib import cm
def plotter_gradcam(X, cam, seg, cam_nr, modality, y_true, y_pred, sample_nr, slice_x, slice_y, slice_z, alpha, threshold, cmap, reverse_cmap):
     
    # Pick and normalize image
    X = np.copy(X[sample_nr])
    X[...,0] = np.divide(X[...,0],np.amax(X[...,0]))
    if modality>0:
        X[...,1] = np.divide(X[...,1],np.amax(X[...,1]))
    
    # Pick cam
    cam = np.copy(cam[cam_nr,sample_nr,...])
    
    # Pick segmentation
    seg = np.copy(seg[sample_nr])
    
    # Colormap
    color_map = plt.cm.get_cmap(cmap)
    if reverse_cmap:
        color_map = color_map.reversed()
        
    print('Image {:d}: [{:.2f}, {:.2f}]'.format(sample_nr,np.amin(X),np.amax(X)))
    print('Cam   {:d}: [{:.2f}, {:.2f}]'.format(sample_nr,np.amin(cam),np.amax(cam)))
    print('True label: {:d}'.format(y_true[sample_nr]))
    print('Prediction: {:.2f}'.format(y_pred[sample_nr]))
    
    # Threshold
    #X_mask = np.zeros_like(X[...,modality])
    #X_mask[np.where(cam>=threshold)]=1
    #X_in = np.copy(X)
    #X_in[...,modality] = np.multiply(X_in[...,modality], X_mask)
    X_in = np.zeros_like(X[...,modality])
    X_in[np.where(cam>=threshold)]=1
    
    #X_mask = np.zeros_like(X[...,modality])
    #X_mask[np.where(cam<threshold)]=1
    #X_out = np.copy(X)
    #X_out[...,modality] = np.multiply(X_out[...,modality], X_mask)
    X_out = np.zeros_like(X[...,modality])
    
    # Figure
    fig, ax = plt.subplots(2, 3, figsize=(14,7.6), sharex=True, sharey=True)
    
    # Original input image with GradCAM on top
    ax[0,0].imshow(X[slice_x,:,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())
    ax[0,1].imshow(X[:,slice_y,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())
    ax[0,2].imshow(X[:,:,slice_z,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())

    ax[0,0].imshow(cam[slice_x,:,:], cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    ax[0,1].imshow(cam[:,slice_y,:], cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    ax[0,2].imshow(cam[:,:,slice_z], cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    
    ax[0,0].set_xlabel('y', fontsize=16)
    ax[0,0].set_ylabel('z', fontsize=16)
    ax[0,1].set_xlabel('x', fontsize=16)
    ax[0,1].set_ylabel('z', fontsize=16)
    ax[0,2].set_xlabel('x', fontsize=16)
    ax[0,2].set_ylabel('y', fontsize=16)
    
    im = plt.imshow(np.zeros((50,50)), cmap=color_map, vmin=0.0, vmax=1.0, alpha=1.0)
    
    # Binarized CT
    ax[1,0].imshow(X[slice_x,:,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())
    ax[1,1].imshow(X[:,slice_y,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())
    ax[1,2].imshow(X[:,:,slice_z,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed())
    
    ax[1,0].imshow(X_in[slice_x,:,:], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    ax[1,1].imshow(X_in[:,slice_y,:], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    ax[1,2].imshow(X_in[:,:,slice_z], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    
    ax[1,0].imshow(X_out[slice_x,:,:], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    ax[1,1].imshow(X_out[:,slice_y,:], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    ax[1,2].imshow(X_out[:,:,slice_z], cmap=plt.cm.get_cmap('bwr'), alpha=0.6)
    
    ax[1,0].set_xlabel('y', fontsize=16)
    ax[1,0].set_ylabel('z', fontsize=16)
    ax[1,1].set_xlabel('x', fontsize=16)
    ax[1,1].set_ylabel('z', fontsize=16)
    ax[1,2].set_xlabel('x', fontsize=16)
    ax[1,2].set_ylabel('y', fontsize=16)
    
    # Settings
    for axis in ax.flatten():
        axis.tick_params(labelsize=14)
    
    plt.tight_layout()
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.8, 0.09, 0.025, 0.885])
    fig.colorbar(im, cax=cbar_ax)
    cbar_ax.tick_params(labelsize=14)
    
    plt.show()

In [ ]:
interactive(plotter_gradcam,
            X            = fixed(X_test),
            cam          = fixed(cam_ensemble),
            seg          = fixed(X_seg),
            y_true       = fixed(y_true),
            y_pred       = fixed(y_pred_ensemble),
            sample_nr    = (0,X_test.shape[0]-1),
            cam_nr       = (0,cam_ensemble.shape[0]-1),
            modality     = (0,X_test.shape[4]-1),
            slice_x      = (0,X_test.shape[1]-1),
            slice_y      = (0,X_test.shape[2]-1),
            slice_z      = (0,X_test.shape[3]-1),
            alpha        = (0.0,1.0),
            threshold    = (0.0,1.0,0.05),
            cmap         = ["inferno", "cool", "gist_yarg", "jet", "magma", "plasma", "viridis"],
            reverse_cmap = [False, True])

### CAM score

In [ ]:
cam_thr = 0.25

In [ ]:
binary_cam_ensemble = np.copy(cam_ensemble)
binary_cam_ensemble[cam_ensemble>=cam_thr]=1.0
binary_cam_ensemble[cam_ensemble<cam_thr]=0.0
binary_cam_ensemble.shape

In [ ]:
seg_size     = [np.where(X_seg[i]==1.0)[0].shape[0] for i in range(X_test.shape[0])]
act_seg_size = [np.sum(binary_cam_ensemble[0,i][np.where(X_seg[i]==1.0)]).astype(np.int32) for i in range(X_test.shape[0])]
cam_score    = np.asarray(act_seg_size).astype(np.float32) / np.asarray(seg_size).astype(np.float32)
cam_score.shape

In [ ]:
print('MyAwesomeScore = {:.2f} +/- {:.2f}'.format(np.median(cam_score), np.percentile(cam_score, 10)))

#### CAM score plots

In [ ]:
# Histogram parameters
bins     = 50
binrange = [0.0,1.0]

# Model gaussian distribution
x_cs          = np.linspace(np.min(cam_score)*0.9,np.max(cam_score)*1.1,num=100)
normal_pdf_cs = norm.pdf(x_cs, loc=np.mean(cam_score), scale=np.std(cam_score))
normal_pdf_cs = normal_pdf_cs / np.amax(normal_pdf_cs)
counts_cs, _  = np.histogram(cam_score, bins=bins, range=binrange)
normal_pdf_cs = normal_pdf_cs * np.amax(counts_cs)

In [ ]:
# Plot histogram and gaussian distribution
plt.figure(figsize=(12,6))
sns.histplot(cam_score, bins=bins, binrange=binrange, color='cornflowerblue', label=('CAM score = {:.2f}+/-{:.2f}'.format(np.mean(cam_score), np.std(cam_score))))
#ax[0].plot(x_cs, normal_pdf_cs, c='red')
plt.axvline(x=np.mean(cam_score), ls='--', lw=2, c='black')
plt.axvline(x=np.mean(cam_score)+1*np.std(cam_score), ls='--', lw=1, c='black', label='Mean +/- SD')
plt.axvline(x=np.mean(cam_score)-1*np.std(cam_score), ls='--', lw=1, c='black')
plt.xlabel('CAM score', fontsize=16)
plt.ylabel('count', fontsize=16)
plt.legend(loc=2, fontsize=14)
plt.tick_params(labelsize=14)
plt.tight_layout()
plt.show()

### Correct and false predictions

In [ ]:
y_pred_ensemble_binary = [0 if pred<0.5 else 1 for pred in y_pred_ensemble]

correct_predictions = np.where(y_true==y_pred_ensemble_binary)
wrong_predictions   = np.where(y_true!=y_pred_ensemble_binary)

pred_result = np.empty(shape=(X_test.shape[0]), dtype=object)
pred_result[correct_predictions] = 'correct'
pred_result[wrong_predictions]   = 'wrong'

correct_predictions, wrong_predictions

In [ ]:
plt.figure(figsize=(9,7))
sns.boxplot(x=pred_result, y=cam_score, palette="Set2")
plt.xlabel("prediction correct", fontsize=16)
plt.ylabel("CAM score", fontsize=16)
plt.tick_params(labelsize=14)
plt.tick_params(labelsize=14)
plt.show()

## Paper

### Figure 4 - GradCAM

In [ ]:
df_info['histology'][df_info['histology']==10]='hyperlastisch'
df_info['histology'][df_info['histology']==17]='regulär'
df_info['histology'][df_info['histology']==11]='lipomatös'
df_info['histology'][df_info['histology']==14]='tubulovillös'
df_info['histology'][df_info['histology']==16]='tubulovillös'
df_info['histology'][df_info['histology']==13]='tubulär'
df_info['histology'][df_info['histology']==15]='villös'
df_info['histology'][df_info['histology']==1] ='adenocarcinoma'
df_info['histology'][df_info['histology']==12]='adenomatous'
histo_label = df_info['histology'].values
histo_label

In [ ]:
np.where(histo_label=='adenocarcinoma')

In [ ]:
def plotter_fig4(X, cam, seg, cam_nr, modality, y_true, y_pred, histo_label, sample_a, sample_b, sample_c, slice_z_a, slice_z_b, slice_z_c, alpha, cmap, reverse_cmap, save):
     
    # Normalize image
    X = np.copy(X)
    X[...,0] = np.divide(X[...,0],np.amax(X[...,0]))
    if modality>0:
        X[...,1] = np.divide(X[...,1],np.amax(X[...,1]))
    
    # Colormap
    color_map = plt.cm.get_cmap(cmap)
    if reverse_cmap:
        color_map = color_map.reversed()
    
    print('A:')
    print('\tImage   {:d}: [{:.2f}, {:.2f}]'.format(sample_a,np.amin(X[sample_a]),np.amax(X[sample_a])))
    print('\tCam     {:d}: [{:.2f}, {:.2f}]'.format(sample_a,np.amin(cam[cam_nr,sample_a]),np.amax(cam[cam_nr,sample_a])))
    print('\tTrue label:  {:d}'.format(y_true[sample_a]))
    print('\tPrediction:  {:.2f}'.format(y_pred[sample_a]))
    print('\tHisto label: {:s}'.format(histo_label[sample_a]))
    
    print('B:')
    print('\tImage   {:d}: [{:.2f}, {:.2f}]'.format(sample_b,np.amin(X[sample_b]),np.amax(X[sample_b])))
    print('\tCam     {:d}: [{:.2f}, {:.2f}]'.format(sample_b,np.amin(cam[cam_nr,sample_b]),np.amax(cam[cam_nr,sample_b])))
    print('\tTrue label:  {:d}'.format(y_true[sample_b]))
    print('\tPrediction:  {:.2f}'.format(y_pred[sample_b]))
    print('\tHisto label: {:s}'.format(histo_label[sample_b]))
    
    print('C:')
    print('\tImage   {:d}: [{:.2f}, {:.2f}]'.format(sample_c,np.amin(X[sample_c]),np.amax(X[sample_c])))
    print('\tCam     {:d}: [{:.2f}, {:.2f}]'.format(sample_c,np.amin(cam[cam_nr,sample_c]),np.amax(cam[cam_nr,sample_c])))
    print('\tTrue label:  {:d}'.format(y_true[sample_c]))
    print('\tPrediction:  {:.2f}'.format(y_pred[sample_c]))
    print('\tHisto label: {:s}'.format(histo_label[sample_c]))
    
    # Figure
    fig, ax = plt.subplots(1, 3, figsize=(18,6), sharey=True)
    
    im = plt.imshow(np.zeros((50,50)), cmap=color_map, vmin=0.0, vmax=1.0, alpha=1.0)
    
    # Original input image
    #ax[0,0].imshow(X[sample_a,:,slice_z_a,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    #ax[0,1].imshow(X[sample_b,:,:,slice_z_b,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    #ax[0,2].imshow(X[sample_c,slice_z_c,:,:,modality], cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    
    # Original input image with GradCAM on top
    ax[0].imshow(np.rot90(X[sample_a,:,slice_z_a,:,modality],k=3,axes=(0,1)), cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    ax[1].imshow(np.rot90(X[sample_b,:,:,slice_z_b,modality],k=3,axes=(0,1)), cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    ax[2].imshow(np.rot90(X[sample_c,slice_z_c,:,:,modality],k=3,axes=(0,1)), cmap=plt.cm.get_cmap('gist_yarg').reversed(), vmin=0.0, vmax=1.0)
    
    ax[0].imshow(np.rot90(cam[cam_nr,sample_a,:,slice_z_a,:],k=3,axes=(0,1)), cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    ax[1].imshow(np.rot90(cam[cam_nr,sample_b,:,:,slice_z_b],k=3,axes=(0,1)), cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    ax[2].imshow(np.rot90(cam[cam_nr,sample_c,slice_z_c,:,:],k=3,axes=(0,1)), cmap=color_map, vmin=0.0, vmax=1.0, alpha=alpha) # overlay
    
    ax[0].set_xlabel('x', fontsize=16)
    ax[0].set_ylabel('y', fontsize=16)
    ax[1].set_xlabel('x', fontsize=16)
    ax[2].set_xlabel('x', fontsize=16)
    
    # Settings
    for axis in ax.flatten():
        axis.tick_params(labelsize=14)
    
    plt.tight_layout()
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.8075, 0.158, 0.025, 0.742])
    fig.colorbar(im, cax=cbar_ax)
    cbar_ax.tick_params(labelsize=14)
    if save:
        plt.savefig('results/cnn_paper/grad_cam_figure.pdf')
    plt.show()

In [ ]:
interactive(plotter_fig4,
            X            = fixed(X_test), #fixed(np.rot90(X_test, axes=(1, 2))),
            cam          = fixed(cam_ensemble), #fixed(np.rot90(cam_ensemble, axes=(2, 3))),
            seg          = fixed(X_seg), #fixed(np.rot90(X_seg, axes=(1, 2))),
            y_true       = fixed(y_true),
            y_pred       = fixed(y_pred_ensemble),
            histo_label  = fixed(histo_label),
            sample_a     = fixed(109), #(0,X_test.shape[0]-1),
            sample_b     = fixed(73), #(0,X_test.shape[0]-1),
            sample_c     = fixed(48), #(0,X_test.shape[0]-1),
            cam_nr       = (0,cam_ensemble.shape[0]-1),
            modality     = (0,X_test.shape[4]-1),
            slice_z_a    = (0,X_test.shape[3]-1),
            slice_z_b    = (0,X_test.shape[3]-1),
            slice_z_c    = (0,X_test.shape[3]-1),
            alpha        = (0.0,1.0),
            cmap         = ["inferno", "cool", "gist_yarg", "jet", "magma", "plasma", "viridis"],
            reverse_cmap = [False, True],
            save         = [False, True])

A: 109 (coronal, 26), B: 73 (axial, 25), C: 48 (saggital, 26) (noch richtig ausrichten)

Zwei versionen: Mit CT & ohne

In [ ]:
df_info.iloc[109]

In [ ]:
df_info.iloc[73]

In [ ]:
df_info.iloc[48]